In [1]:
import sqlite3
import pandas as pd
import csv
from flask import Flask, render_template_string, render_template
from werkzeug.serving import run_simple

df= pd.read_csv("index.csv")
df

,date,datetime,cash_type,card,money,coffee_name
0,2024-03-01,2024-03-01 10:15:50.520,card,ANON-0000-0000-0001,38.70,Latte
1,2024-03-01,2024-03-01 12:19:22.539,card,ANON-0000-0000-0002,38.70,Hot Chocolate
2,2024-03-01,2024-03-01 12:20:18.089,card,ANON-0000-0000-0002,38.70,Hot Chocolate
3,2024-03-01,2024-03-01 13:46:33.006,card,ANON-0000-0000-0003,28.90,Americano
4,2024-03-01,2024-03-01 13:48:14.626,card,ANON-0000-0000-0004,38.70,Latte
...,...,...,...,...,...,...
971,2024-07-14,2024-07-14 22:31:29.976,card,ANON-0000-0000-0376,32.82,Latte
972,2024-07-15,2024-07-15 07:33:05.557,card,ANON-0000-0000-0377,32.82,Cappuccino
973,2024-07-16,2024-07-16 12:23:37.467,card,ANON-0000-0000-0378,27.92,Americano with Milk
974,2024-07-16,2024-07-16 19:29:25.485,card,ANON-0000-0000-0367,32.82,Hot Chocolate


In [2]:
con = sqlite3.connect('coffee_data.db')
cursor = con.cursor()

create_table = """CREATE TABLE IF NOT EXISTS coffee
                                    (ID INTPRIMARY KEY,
                                    Date  DATE,
                                    Date_time DATE,
                                    cash_type TEXT,
                                    card   TEXT,
                                    money float,
                                    coffee name TEXT
                                    );
                                    """
cursor.execute(create_table)
df.to_sql('coffee_data.db', con, if_exists='replace', index=False)

con.commit()
con.close()

In [3]:
app = Flask(__name__,template_folder='.' )

@app.route('/')
def home():
    data_html = df.head(10).to_html(index=False)
    return render_template_string('''
    <!DOCTYPE html>
    <html>
    <head> <title>Coffee </title></head>
    <body>
    <h1>We Are Welcoming You To Our Coffee </h1>
    <nav>
    <div>{{ data|safe }}</div>
     <ul> 
          <li><a href="/"><button>Home</button></a></li> 
          <li><a href="/about"><button>About</button></a></li>
          <li><a href="/data"><button>Data</button></a></li>
        </ul>
    </nav>
</body>
</html>
''', data=data_html)

@app.route('/about')
def about():
    return render_template_string('''
    <!DOCTYPE html>
<html>
<head>
    <title>About</title>
</head>
<body>
    <h1>About Coffee</h1>
    <p>Source: <a href="https://www.kaggle.com/datasets/ihelon/coffee-sales">Coffee Sales Dataset</a></p>
    <p>Type of variables:</p>
    <ul>
        <li>Date: Date : coffee sale</li>
        <li>Date_time: Date :sailing time for coffee</li>
        <li>cash_type: Text : price for coffee</li>
        <li>card: Text : giving money this card</li>
        <li>money: INT :price</li>
        <li>coffee_name: Text : name of coffee</li>
    </ul>
</body>
</html>
''')

@app.route('/data')

def data():
    con = sqlite3.connect('coffee_data.db')
    c = con.cursor()
    c.execute('SELECT * FROM coffee LIMIT 10')  
    data = c.fetchall()
    con.close()
    
    return render_template_string('''<!DOCTYPE html>
    <html>
    <body>
    <h1>Data from Coffee Database</h1>
    <table border="1">
        <tr>
            <th>Date</th>
            <th>Date_time</th>
            <th>cash_type</th>
            <th>card</th>
            <th>money</th>
            <th>coffee_name</th>
        </tr>
        {% for row in data %}
        <tr>
            {% for item in row %}
            <td>{{ item }}</td>
            {% endfor %}
        </tr>
        {% endfor %}
    </table>
    </body>
    </html>''', data=data)

In [ ]:
run_simple('localhost',9200,app,use_reloader=False, use_debugger=False)

 * Running on http://localhost:9200
Press CTRL+C to quit
127.0.0.1 - - [22/Jul/2024 18:39:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2024 18:39:13] "GET /data HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2024 18:39:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2024 18:39:27] "GET /about HTTP/1.1" 200 -
